<a href="https://colab.research.google.com/github/ftmthb/NER/blob/main/NER_fine_tune_classes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys
sys.path.append("/content/drive/MyDrive/virtual_env/lib/python3.10/site-packages")

### Data import
+only taking dcts with annotations, clean,

In [8]:
def import_func(path):
  ext = str(path).split('.')[-1]
  if ext == 'json':
    import json
    with open(path) as file:
      raw_data = json.load(file)
  elif ext == 'pkl':
    import pickle
    with open(path, 'rb') as file:
      raw_data = pickle.load(file)
  else:
    raise ValueError("Unsupported file format. Only JSON (.json) and Pickle (.pkl) files are supported.")
  return raw_data
train_data = import_func(r"/content/drive/MyDrive/ner_india/NER_TRAIN_JUDGEMENT.json")
dev_data = import_func(r"/content/drive/MyDrive/ner_india/NER_DEV_PREAMBLE.json")
# data[0].keys()

#### draft

In [ ]:
import json

with open(r"/content/drive/MyDrive/ner_india/NER_TRAIN_JUDGEMENT.json") as file:
    json_data = json.load(file)
len(json_data)

9435

In [ ]:
import json
with open(r"/content/drive/MyDrive/ner_india/NER_DEV_PREAMBLE.json") as file:
    dev_data = json.load(file)
print(len(dev_data))

125


In [9]:
train_data[3]['annotations'] == [{'result': []}]

True

In [ ]:
for indice in range(len(json_data)):
  for annot in json_data[indice]['annotations']:
    if annot['result']== []:
      print(indice)
      break #3, 6, 8, 16

3
6
8
16
19
22
23
29
32
38
40
42
45
48
66
67
69
76
80
81
87
89
90
99
108
109
110
114
118
119
122
125
127
128
129
132
133
134
137
146
147
150
152
154
161
164
169
173
178
183
190
192
194
196
199
200
203
204
210
211
218
220
225
228
234
235
241
255
256
259
260
261
265
266
271
278
280
288
292
293
295
296
305
306
316
317
324
326
350
361
364
370
373
381
382
387
392
396
397
405
406
411
414
418
423
425
431
432
441
442
444
450
459
462
464
465
468
480
487
489
500
507
508
530
543
545
546
547
549
555
561
567
572
579
585
621
624
626
627
630
636
637
639
644
645
646
647
650
654
659
661
664
675
676
679
688
702
722
727
731
732
743
745
746
750
758
761
768
771
772
778
780
784
787
793
801
804
805
806
809
814
825
829
838
839
849
859
871
879
880
883
889
890
895
901
903
906
907
916
919
920
921
922
927
929
934
937
941
944
955
960
962
974
977
980
990
991
992
998
1000
1011
1013
1016
1019
1022
1023
1036
1039
1055
1063
1069
1074
1077
1088
1089
1094
1103
1109
1113
1117
1118
1122
1137
1141
1143
1144
1147
1148
1153
1

### DataPrep

In [123]:
class DataPrep:

  """
    The role of this class is to prepare the raw documents, it has three functions:
      * clean_text_data
      * create_text_data
      * get_new_position
      detailled bellow
  """

#perhaps create a function that actually check for other characters that will be tokenized into whitespace or other special characters

  def clean_text_data(text_data):

    """
      the role of this function is to get rid of the special charcters that are either tokenized into whitespace like '\xad',
      or that will take space into the tokens vector like the succession of '===' where each one is a single token
      Every whitespace will be left to the tokenizer to deal with and the following code is set based on that
    """

    import re
    replacements = [
        (r'\.{2,}', ' '),
        (r'\={2,}', ' '),
        (r'\-{2,}', ' '),
        (r'\xad', ' '),
        (r'\xa0', ' '),
        (r'\x80', ''),
        (r'\x9d', ''),
        (r'\x13', ' '),
        ]

    for old, new in replacements:
      text_data = re.sub(old, new, text_data)
    return text_data


  def create_text_data(raw_json_data):

    """
      this function creates a vector for each document using the cleaning function
    """

    the_text_data = []
    for line in raw_json_data:
      # if line['annotations'] != [{'result': []}]:   # the document has annotations  #in fact, we'll clean the whole data and leave the test to be done later
      the_text_data.append(DataPrep.clean_text_data(line['data']['text']))
    return the_text_data

  def get_new_position(old_text, new_text, pos):
    import re


    """
      this function will allow to find the position of a special entity in the new cleaned text using the position in the raw text given in annotations
    """

    if new_text[pos] == old_text[pos]: #pos did not change
      return pos

    else: # we actually need to update the pos

      special_entity = old_text[pos]
      if DataPrep.clean_text_data(special_entity).split() == []:
        # print(special_entity, pos)
        return 'not so special entity'      #unnecessary prints, todo: remove this print, it's basically used to test the output later: type ?= slice

      if special_entity[0:2] == '--':
        special_entity = special_entity[2:] #cases where the NE starts with -, preceeded by -- in data['text'] and omitted by the cleaninh function
        pos = slice(pos.start + 2, pos.stop)

      if special_entity[0:1] == '-' and special_entity[1:2] != '-':
        special_entity = special_entity[1:] #cases where the NE starts with -, preceeded by -- in data['text'] and omitted by the cleaninh function
        pos = slice(pos.start + 1, pos.stop)

      in_old_text = [m.start() for m in re.finditer(re.escape(special_entity), old_text)]
      in_new_text = [m.start() for m in re.finditer(re.escape(DataPrep.clean_text_data(special_entity)), new_text)]

      """
        the previous lists take into consideration the possibility of having a special entity with more than on occurence,
        so they'll make sure to align the occurences in the raw data and in the clean version
      """

      if len(in_new_text) != len(in_old_text) and special_entity != '.':  #written differently #with whitespaces, or special characters
        # this block is for test, and shouldn't be achieved
        # print(indice, "\n not same len")      #unnecessary prints, todo: remove this print
        print('\t special_entity: ', special_entity, 'at pos', pos)
        print('in_old_text: ', in_old_text, '\t in_new_text: ',in_new_text)

      else:

        all_occ = dict(zip(in_old_text,in_new_text))

        start = all_occ[pos.start]

        end = start+len(DataPrep.clean_text_data(old_text[pos]))#-1

        # check
        if DataPrep.clean_text_data(old_text[pos]) != new_text[slice(start, end)]:
          print('not similar', old_text[pos], '\t', new_text[slice(start, end)])  #unnecessary prints, todo: remove this print


        return slice(start,end)


  # dev_text_data = create_text_data(dev_data)
  # train_text_data =create_text_data(json_data)

In [90]:
train_text_data = DataPrep.create_text_data(train_data)
dev_text_data = DataPrep.create_text_data(dev_data)
train_text_data[0]

"\n\n(7) On specific query by the Bench about an entry of Rs. 1,31,37,500 on deposit side of Hongkong Bank account of which a photo copy is appearing at p. 40 of assessee's paper book, learned authorised representative submitted that it was related to loan from broker, Rahul & Co. on the basis of his submission a necessary mark is put by us on that photo copy."

In [91]:
def set_model_tokenize(model_name, train_text_data, dev_text_data ):
  #set model (name) & tokenize train and dev data
  model = model_name
  from transformers import AutoTokenizer
  tokenizer = AutoTokenizer.from_pretrained(model)

  train_tokenized = tokenizer(train_text_data,  is_split_into_words = False, return_offsets_mapping = True, add_special_tokens = True, truncation = True, padding = 'max_length', max_length=512, stride=128, return_overflowing_tokens=True,
  )  #when text is not splitted, the positions in str are accurate

  train_tokens = [tokenizer.convert_ids_to_tokens(tokenized) for tokenized in train_tokenized.input_ids]

  dev_tokenized = tokenizer(dev_text_data,  is_split_into_words = False,  return_offsets_mapping = True, add_special_tokens = True,  padding = 'max_length', truncation = True, max_length=512, stride=128, return_overflowing_tokens=True, )  #when text is not splitted, the positions are accurate

  dev_tokens = [tokenizer.convert_ids_to_tokens(tokenized) for tokenized in dev_tokenized.input_ids]

  return train_tokenized, dev_tokenized, train_tokens, dev_tokens


In [18]:
def get_token_pos(mappers, start_str, end_str):
  # get position in tokens list from annotations
  for idx, (start, end) in enumerate(mappers):
    if idx<= mappers.index(max(mappers)):
        if start <= start_str:
            idx_start = idx
        if end == end_str:
            idx_end = idx+1
        if end < end_str:
            idx_end = idx+2

  pos = slice(idx_start, idx_end)
  return pos

In [147]:
# I still need to include cases where there are no annotations, this wouldn't be aligned actually because
# I think I did with         # if result != []:   but still need to check the alignment  too

def affect(raw_json_data, the_tokenized, the_text_data, the_tokens):

  the_tokens_labels = [['O' for mapper in the_tokenized["offset_mapping"][i]] for i in range(len(the_tokenized["offset_mapping"]))]

  for indice in range(len(raw_json_data)):
    # print(indice)
    tkn_indice = the_tokenized['overflow_to_sample_mapping'].index(indice)
    # print('tkn_indice: ', tkn_indice)

    for annot in raw_json_data[indice]['annotations']: #list of all the entities that are part of the deal
      if annot != [{'result': []}]: #is there a risk of mismatch with the txt data prep, append
        for result in annot['result']:    #how does it deal with the empty ones
          deb_str = result['value']['start']
          fin_str = result['value']['end']
          label = result['value']['labels'][0]

          raw_text = raw_json_data[indice]['data']['text']
          special_entity = result['value']['text']
          clean_pos = DataPrep.get_new_position(raw_text,the_text_data[indice], slice(deb_str,fin_str) ) #will give the tupple for the offset_map

          if type(clean_pos) == slice:

            if the_tokenized['overflow_to_sample_mapping'].count(indice) > 1:

            #split into chunks: we need to check into both using offset mapping for example
              for i in range(the_tokenized['overflow_to_sample_mapping'].count(indice)+1) :
                if clean_pos.start > the_tokenized["offset_mapping"][tkn_indice+i+1][2][0] and clean_pos.stop < max(the_tokenized["offset_mapping"][tkn_indice+i+1])[1]:

                  mapper = the_tokenized["offset_mapping"][tkn_indice+i+1]
                  token_pos =  get_token_pos(mapper, clean_pos.start, clean_pos.stop)

                  the_tokens_labels[tkn_indice+i+1][token_pos.start] = 'B-'+label

                  for x in range(token_pos.start+1, token_pos.stop):
                    if x <len(the_tokens_labels[tkn_indice+i+1]):
                      the_tokens_labels[tkn_indice+i+1][x] = 'I-'+label


                if clean_pos.start > the_tokenized["offset_mapping"][tkn_indice+i][2][0] and clean_pos.stop < max(the_tokenized["offset_mapping"][tkn_indice+i])[1]:
                  #update the clean_pos too
                  tkn_indice += i
                  break

            mapper = the_tokenized["offset_mapping"][tkn_indice]
            token_pos =  get_token_pos(mapper, clean_pos.start, clean_pos.stop)

            #check similarity

            if ''.join(the_tokens[tkn_indice][token_pos]).replace('##', '').replace('Ċ', '').replace('Ġ', '') != ''.join(the_text_data[indice][clean_pos].split()) and abs(len(''.join(the_tokens[tkn_indice][token_pos]).replace('##', '').replace('Ċ', '').replace('Ġ', ''))-len(''.join(the_text_data[indice][clean_pos].split())))>5:
              #the length check was added, because sometimes the tokenized might include what seems to be a part of the special entity like ., the year in a date, ).
              print('here?!', indice,tkn_indice, special_entity, the_tokens[tkn_indice][token_pos], '\t', token_pos, '\t', slice(deb_str,fin_str),  clean_pos )


            the_tokens_labels[tkn_indice][token_pos.start] = 'B-'+label

            for x in range(token_pos.start+1, token_pos.stop):
              if x <len(the_tokens_labels[tkn_indice]):
                the_tokens_labels[tkn_indice][x] = 'I-'+label

              else:
                if x!=512 :
                  print('not simillar,  \t', 'indice: \t', indice, '\t x: \t', x )

          else:
            print('not so special entity')

# tokens_labels = affect(tokenized, text_data, json_data, tokens):
# dev_tokens_labels = affect(dev_tokenized, dev_text_data, dev_data, dev_tokens):


#### draft

In [32]:
# model_name = 'roberta-base'
model_name = 'dslim/bert-base-NER'
train_tokenized, dev_tokenized, train_tokens, dev_tokens = set_model_tokenize(model_name, train_text_data, dev_text_data )


In [93]:
model_name = 'roberta-base'
# model_name = 'dslim/bert-base-NER'
rob_train_tokenized, rob_dev_tokenized, rob_train_tokens, rob_dev_tokens = set_model_tokenize(model_name, train_text_data, dev_text_data )


In [30]:
train_tokenized.keys()  #[tokenizer.convert_ids_to_tokens(tokenized) for tokenized in train_tokenized.input_ids]

dict_keys(['input_ids', 'attention_mask', 'offset_mapping', 'overflow_to_sample_mapping'])

In [35]:
train_tokens[0][:10]

['[CLS]', '(', '7', ')', 'On', 'specific', 'que', '##ry', 'by', 'the']

In [36]:
rob_train_tokens[0][:10]


['<s>', 'Ċ', 'Ċ', '(', '7', ')', 'ĠOn', 'Ġspecific', 'Ġquery', 'Ġby']

In [56]:
# print(train_tokenized['input_ids'][0][:100])
train_tokenized.keys()
for tokenized in train_tokenized['input_ids'][0][:10]:
  print(tokenized)

101
113
128
114
1212
2747
15027
1616
1118
1103


In [58]:
print([AutoTokenizer.from_pretrained('roberta-base').convert_ids_to_tokens(tokenized) for tokenized in rob_train_tokenized['input_ids'][0][:100]])

['<s>', 'Ċ', 'Ċ', '(', '7', ')', 'ĠOn', 'Ġspecific', 'Ġquery', 'Ġby', 'Ġthe', 'ĠBench', 'Ġabout', 'Ġan', 'Ġentry', 'Ġof', 'ĠRs', '.', 'Ġ1', ',', '31', ',', '37', ',', '500', 'Ġon', 'Ġdeposit', 'Ġside', 'Ġof', 'ĠHong', 'k', 'ong', 'ĠBank', 'Ġaccount', 'Ġof', 'Ġwhich', 'Ġa', 'Ġphoto', 'Ġcopy', 'Ġis', 'Ġappearing', 'Ġat', 'Ġp', '.', 'Ġ40', 'Ġof', 'Ġass', 'essee', "'s", 'Ġpaper', 'Ġbook', ',', 'Ġlearned', 'Ġauthorised', 'Ġrepresentative', 'Ġsubmitted', 'Ġthat', 'Ġit', 'Ġwas', 'Ġrelated', 'Ġto', 'Ġloan', 'Ġfrom', 'Ġbroker', ',', 'ĠRahul', 'Ġ&', 'ĠCo', '.', 'Ġon', 'Ġthe', 'Ġbasis', 'Ġof', 'Ġhis', 'Ġsubmission', 'Ġa', 'Ġnecessary', 'Ġmark', 'Ġis', 'Ġput', 'Ġby', 'Ġus', 'Ġon', 'Ġthat', 'Ġphoto', 'Ġcopy', '.', '</s>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']


In [63]:
# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained('roberta-base')

# Get the first 100 token IDs
token_ids = rob_train_tokenized['input_ids'][7][:100]

# Convert token IDs to tokens
tokens = tokenizer.convert_ids_to_tokens(token_ids)

# Convert tokens to string
tokens_string = tokenizer.convert_tokens_to_string(tokens)

# Print the result
# print(tokens_string)
for token_id, token in zip(token_ids, tokens):
    print(f"Token ID: {token_id}, Token: {token}")

Token ID: 0, Token: <s>
Token ID: 4148, Token: On
Token ID: 5, Token: Ġthe
Token ID: 363, Token: Ġnight
Token ID: 9, Token: Ġof
Token ID: 971, Token: Ġ28
Token ID: 494, Token: ĠMarch
Token ID: 6, Token: ,
Token ID: 23342, Token: Ġ1959
Token ID: 6, Token: ,
Token ID: 28606, Token: ĠKrish
Token ID: 8697, Token: nam
Token ID: 710, Token: ur
Token ID: 212, Token: th
Token ID: 118, Token: i
Token ID: 17347, Token: ĠRao
Token ID: 6, Token: ,
Token ID: 309, Token: Ġaccording
Token ID: 7, Token: Ġto
Token ID: 39, Token: Ġhis
Token ID: 527, Token: Ġstory
Token ID: 6, Token: ,
Token ID: 3203, Token: Ġwalked
Token ID: 88, Token: Ġinto
Token ID: 5, Token: Ġthe
Token ID: 929, Token: Ġroom
Token ID: 9, Token: Ġof
Token ID: 5, Token: Ġthe
Token ID: 31390, Token: Ġpetitioner
Token ID: 6, Token: ,
Token ID: 851, Token: Ġgave
Token ID: 5, Token: Ġthe
Token ID: 25677, Token: Ġbribe
Token ID: 8, Token: Ġand
Token ID: 156, Token: Ġmade
Token ID: 5, Token: Ġthe
Token ID: 1198, Token: Ġpre
Token ID: 12, Toke

In [47]:
from transformers import AutoTokenizer

[AutoTokenizer.from_pretrained('roberta-base').convert_ids_to_tokens(tokenized) for tokenized in train_tokenized['input_ids'][0][:100]]
# rob_train_tokenized[0]

['Ġlike',
 '"',
 "Ġ'",
 'Ġif',
 'Ġland',
 'AR',
 'ither',
 'Ġopportunities',
 'Ġcompared',
 'Ġcharges',
 'Ġcontest',
 'Ġversion',
 'Ġpressure',
 'Ġdebt',
 'ĠWh',
 'Ġwhite',
 'human',
 'm',
 'Ġnow',
 'Ġno',
 'Ġfigure',
 'Ġno',
 'Ġbuildings',
 'Ġno',
 'Ġmostly',
 'ana',
 'Ġpetroleum',
 'ter',
 'Ġwhite',
 'Ġheaded',
 'ĠFilipino',
 'Ġself',
 'Ġfamiliar',
 'Ġfill',
 'Ġwhite',
 'Ġgroups',
 'We',
 'Ġsettle',
 'Ġteammates',
 'Ġterms',
 'Ġcrews',
 'ard',
 'Ġtake',
 'm',
 'Ġperfect',
 'Ġwhite',
 'ĠMirror',
 'Ġwalking',
 'Ġ1',
 'ĠNew',
 'Ġstandard',
 'Ġbanks',
 'Ġno',
 'ĠCredit',
 'Ġamassed',
 'ĠAffairs',
 'Ġdivorce',
 'ap',
 'Ġsimilar',
 'Ġbreak',
 'Ġgreen',
 'If',
 'esh',
 'In',
 'Ġrusher',
 'Ġno',
 'Ġfairness',
 'Ġadmiration',
 'Ġ-',
 'Ġbrings',
 'm',
 'ana',
 'Ġcharges',
 'Ġveteran',
 'Ġwhite',
 'Ġnine',
 'Ġfamously',
 'We',
 'Ġepisode',
 'Ġfailing',
 'Ġterms',
 'Ġmind',
 'Ġcompared',
 '18',
 'ana',
 'ap',
 'Ġsettle',
 'Ġteammates',
 'm',
 'a',
 '<s>',
 '<s>',
 '<s>',
 '<s>',
 '<s>',
 '<s>',


In [45]:
train_tokenized['input_ids'][0]

[101,
 113,
 128,
 114,
 1212,
 2747,
 15027,
 1616,
 1118,
 1103,
 3096,
 1732,
 1164,
 1126,
 3990,
 1104,
 19003,
 119,
 122,
 117,
 1955,
 117,
 3413,
 117,
 2260,
 1113,
 14304,
 1334,
 1104,
 3475,
 19890,
 1403,
 2950,
 3300,
 1104,
 1134,
 170,
 6307,
 5633,
 1110,
 5452,
 1120,
 185,
 119,
 1969,
 1104,
 15187,
 3051,
 112,
 188,
 2526,
 1520,
 117,
 3560,
 21812,
 4702,
 7402,
 1115,
 1122,
 1108,
 2272,
 1106,
 4891,
 1121,
 24535,
 117,
 16890,
 24287,
 111,
 3291,
 119,
 1113,
 1103,
 3142,
 1104,
 1117,
 13455,
 170,
 3238,
 4551,
 1110,
 1508,
 1118,
 1366,
 1113,
 1115,
 6307,
 5633,
 119,
 102,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 

#### affect test

In [148]:
train_tokens_labels = affect(train_data, rob_train_tokenized, train_text_data, rob_train_tokens)

 slice(112, 113, None)
not so special entity
here?! 1068 1070 Union of India                           v. Jai Narayan Singh,  1995 Supp. (4) 672 ['.', '</s>'] 	 slice(46, 48, None) 	 slice(420, 502, None) slice(420, 502, None)
here?! 1068 1070 State of H.P. v. Suresh Kumar Verma ['.', '</s>'] 	 slice(46, 48, None) 	 slice(707, 742, None) slice(707, 742, None)
here?! 5824 5825 --'Paupuk Kannu Anni v. Thoppayya Mudaliar', (J) :   ["Ġ'", 'P', 'au', 'p', 'uk', 'ĠK', 'ann', 'u', 'ĠAnn', 'i', 'Ġv', '.', 'ĠTh', 'opp', 'ay', 'ya', 'ĠMud', 'ali', 'ar', "',", 'Ġ(', 'J', ')', 'Ġ:', 'Ġ', 'ĠClause'] 	 slice(14, 40, None) 	 slice(38, 90, None) slice(39, 89, None)


In [168]:
train_text_data[1068][420:502]

'Union of India                           v. Jai Narayan Singh,  1995 Supp. (4) 672'

In [166]:
for indice in range(len(train_text_data)):
  if rob_train_tokenized['overflow_to_sample_mapping'].count(indice) > 1:
    print(indice)
rob_train_tokenized['overflow_to_sample_mapping'].count(1068), rob_train_tokenized['overflow_to_sample_mapping'].index(1068)

1068


(2, 1068)

In [173]:
clean_pos = slice(420, 502)
tkn_indice = 1068
i=0
clean_pos.start > rob_train_tokenized["offset_mapping"][tkn_indice+i+1][2][0], clean_pos.stop < max(rob_train_tokenized["offset_mapping"][tkn_indice+i+1])[1]


(False, True)

In [163]:
len(train_text_data), len(rob_train_tokens)

(9435, 9436)

In [161]:
# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained('roberta-base')

# Get the first 100 token IDs
my_token_ids = rob_train_tokenized['input_ids'][1068]#[:100]

# Convert token IDs to tokens
my_tokens = tokenizer.convert_ids_to_tokens(token_ids)

# Convert tokens to string
my_tokens_string = tokenizer.convert_tokens_to_string(tokens)

# Print the result
# print(tokens_string)
# for token_id, token in zip(my_token_ids, my_tokens):
#     print(f"Token ID: {token_id}, Token: {token}")
my_tokens_string#[420:502]

'<s>On the night of 28 March, 1959, Krishnamurthi Rao, according to his story, walked into the room of the petitioner, gave the bribe and made the pre-arranged signal which brought the Assistant Superintendent of Police and his companions to the room.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>'

In [85]:
train_data[4]['data']['text'], train_text_data[3]

('The pillion rider T.V. Satyanarayana Murthy also sustained injuries.',
 'The pillion rider T.V. Satyanarayana Murthy also sustained injuries.')

In [24]:
# train_tokenized["offset_mapping"][0]
kk = [['O' for mapper in train_tokenized["offset_mapping"][i]] for i in range(len(train_tokenized["offset_mapping"]))]

In [ ]:
def update_ne_dist(the_tokens_labels, ne_distribution ):
  for labels in the_tokens_labels:
    for idx in range(len(labels)):
      if labels[idx] in ne_distribution.keys():
        ne_distribution[labels[idx]] +=1
      else:
        ne_distribution[labels[idx]] = 1
  return ne_distribution

ne_distribution = {}
ne_distribution = update_ne_dist(tokens_labels, ne_distribution)
ne_distribution = update_ne_dist(dev_tokens_labels, ne_distribution)

labels_list = list(ne_distribution.keys())


In [ ]:
def get_int_tokens_lab(the_tokens_labels):
  import copy
  the_int_tokens_labels = copy.deepcopy(the_tokens_labels)

  for i in range(len(the_tokens_labels)):
    for j in range(len(the_tokens_labels[i])):
      the_int_tokens_labels[i][j] = int_labels[the_tokens_labels[i][j]]

int_tokens_labels = get_int_tokens_lab(tokens_labels)
int_tokens_labels = get_int_tokens_lab(dev_tokens_labels)

In [ ]:
#appearently I used the with_snnot_id in data gen

In [ ]:
def dtst_gen(the_json_data, the_int_tokens_labels, the_tokenized):
  for id, content in enumerate(the_json_data): #content is basically the dct with its annotations
    if content['annotations'] == [{'result': []}]:  #following the same logic and only use the annotated data
      yield {'labels' : the_int_tokens_labels[id], 'input_ids': the_tokenized['input_ids'][id] , 'attention_mask': the_tokenized['attention_mask'][id]}

# from datasets import Dataset
# no_ne_gen = lambda: dtst_gen(json_data, int_tokens_labels, tokenized) #we need to define it using lambda otherwise we might immediately call the generator
# with_ne_train_dt = Dataset.from_generator(no_ne_gen)

# dev_gen = lambda: dtst_gen(dev_data, dev_int_tokens_labels, dev_tokenized)
# dev_dt = Dataset.from_generator(dev_gen)


In [ ]:
class Pre_processing:
    def __init__(self):

    def drop(self, data, drop_strategies):
        return self.data

    def fillna(self, ntrain, fill_strategies):
        def fill(column, fill_with):

    def feature_engineering(self):

    def label_encoder(self, columns):

    def get_dummies(self, columns):

    def norm_data(self, columns):


In [ ]:
class Pre_processing:
    """
    This class prepares the data berfore applying ML
    """
    def __init__(self):

        print()
        print('pre-processing object is created')
        print()

    def drop(self, data, drop_strategies):
        """
        This function is used to drop a column or row from the dataset.
        ...
        Attributes
        ----------
        data : Pandas DataFrame
            The data you want to drop data from.
        drop_strategies : A list of tuples, each tuple has the data to drop,
        and the axis(0 or 1)

        Returns
        ----------
        A new dataset after dropping the unwanted data.
        """

        self.data=data

        for columns, ax in drop_strategies:
            if len(columns)==1:
                self.data=self.data.drop(labels=column, axis=ax)
            else:
                for column in columns:
                    self.data=self.data.drop(labels=column, axis=ax)
        return self.data

    def fillna(self, ntrain, fill_strategies):
        """
        This function fills NA/NaN values in a specific column using a specified method(zero,mean,...)
        ...
        Attributes
        ----------
        data : Pandas DataFrame
            The data you want to impute its missing values
        fill_strategies : A dictionary, its keys represent the columns,
        and the values represent the value to use to fill the Nulls.

        Returns
        ----------
        A new dataset without null values.
        """
        def fill(column, fill_with):

                if str(fill_with).lower() in ['zero', 0]:
                    self.data[column].fillna(0, inplace=True)
                elif str(fill_with).lower()=='mode':
                    self.data[column].fillna(self.data[column].mode()[0], inplace=True)
                elif str(fill_with).lower()=='mean':
                    self.data[column].fillna(self.data[column].mean(), inplace=True)
                elif str(fill_with).lower()=='median':
                    self.data[column].fillna(self.data[column].median(), inplace=True)
                else:
                    self.data[column].fillna(fill_with, inplace=True)

                return self.data

        #LotFrontage: Linear feet of street connected to property
        self.data['LotFrontage'] = self.data.groupby('Neighborhood')['LotFrontage'].apply(lambda x: x.fillna(x.median())).values

        # Meaning that NO Masonry veneer
        self.data['MSZoning'] = self.data['MSZoning'].transform(lambda x: x.fillna(x.mode().values[0]))

        #imputing columns according to its strategy
        for columns, strategy in fill_strategies:
            if len(columns)==1:
                fill(columns[0], strategy)
            else:
                for column in columns:
                    fill(column, strategy)

        return self.data

    def feature_engineering(self):
        """
        This function is used to apply some feature engineering on the data.
        ...
        Attributes
        ----------
        data : Pandas DataFrame
            The data you want to apply feature engineering on.

        Returns
        ----------
        A new dataset with new columns and some additions.
        """
        # creating new columns
        self.data['TotalSF'] = self.data['TotalBsmtSF'] + self.data['1stFlrSF'] + self.data['2ndFlrSF']

        # Convert some columns from numeric to string
        self.data[['YrSold','MSSubClass','MoSold','OverallCond']] = self.data[['YrSold','MSSubClass','MoSold','OverallCond']].astype(str)

        # Convert some columns from numeric to int
        self.data[['BsmtHalfBath','BsmtFinSF1', 'BsmtFinSF2','BsmtFullBath','BsmtUnfSF','GarageCars','GarageArea']]\
        =self.data[['BsmtHalfBath','BsmtFinSF1', 'BsmtFinSF2','BsmtFullBath','BsmtUnfSF','GarageCars','GarageArea']].astype(int)

        return self.data

    def label_encoder(self, columns):
        """
        This function is used to encode the data to categorical values to benefit from increasing or
        decreasing to build the model
        ...
        Attributes
        ----------
        data : Pandas DataFrame
            The data you want to encode.
        columns : columns to convert.

        Returns
        ----------
        A dataset without categorical data.
        """

        # Convert all categorical collumns to numeric values
        lbl = LabelEncoder()

        self.data[columns] = self.data[columns].apply(lambda x:lbl.fit_transform(x.astype(str)).astype(int))

        return self.data

    def get_dummies(self, columns):
        """
        This function is used to convert the data to dummies values.
        ...
        Attributes
        ----------
        data : Pandas DataFrame
            The data you want to convert.

        Returns
        ----------
        A dataset with dummies.
        """

        # convert our categorical columns to dummies
        for col in columns:
            dumm = pd.get_dummies(self.data[col], prefix = col, dtype=int)
            self.data = pd.concat([self.data, dumm], axis=1)

        self.data.drop(columns, axis=1, inplace=True)

        return self.data

    def norm_data(self, columns):
        """
        This function is used to normalize the data.
        ...
        Attributes
        ----------
        data : Pandas DataFrame
            The data you want to normalize.

        Returns
        ----------
        A new normalized dataset.
        """

        # Normalize our numeric data
        self.data[columns] = self.data[columns].apply(lambda x:np.log1p(x)) #Normalize the data with Logarithms

        return self.data

In [ ]:
class DataPrep():

  def clean_text_data(text_data):
    import re
    replacements = [
        (r'\.{2,}', ' '),
        (r'\={2,}', ' '),
        (r'\-{2,}', ' '),
        (r'\xad', ' '),
        (r'\xa0', ' '),
        (r'\x80', ''),
        (r'\x9d', ''),
        (r'\x13', ' '),
        ]

    for old, new in replacements:
      text_data = re.sub(old, new, text_data)
    return text_data

  def create_text_data(raw_json_data):
    the_text_data = []
    for line in raw_json_data:
      if line['annotations'] != ''

      the_text_data.append(clean_text_data(line['data']['text']))
    return the_text_data



  dev_text_data = create_text_data(dev_data)
  train_text_data =
  plain_text_data = []
  # for line in json_data:
  for idx in with_annot_indices:
      text_data.append(clean_text_data(json_data[idx]['data']['text']))
      plain_text_data.append(json_data[idx]['data']['text'])

  def get_new_position(old_text, new_text, pos):
    #a function that'll allow to find the position of a special entity in the new cleaned text using the position in the raw text given in annotations

    if new_text[pos] == old_text[pos]: #pos did not change
      return pos

    else: # we actually need to update the pos

      special_entity = old_text[pos]
      if clean_text_data(special_entity).split() == []:
        return 'not so special entity'

      if special_entity[0:2] == '--':
        special_entity = special_entity[2:] #cases where the NE starts with -, preceeded by -- in data['text'] and omitted by the cleaninh function
        pos = slice(pos.start + 2, pos.stop)

      if special_entity[0:1] == '-' and special_entity[1:2] != '-':
        special_entity = special_entity[1:] #cases where the NE starts with -, preceeded by -- in data['text'] and omitted by the cleaninh function
        pos = slice(pos.start + 1, pos.stop)

      in_old_text = [m.start() for m in re.finditer(re.escape(special_entity), old_text)]
      in_new_text = [m.start() for m in re.finditer(re.escape(clean_text_data(special_entity)), new_text)]

      if len(in_new_text) != len(in_old_text) and special_entity != '.':  #written differently #with whitespaces, or special characters
        # this block is for test, and shouldn't be achieved
        print(indice, "\n not same len")
        print('\t special_entity: ', special_entity, 'at pos', pos)
        print('in_old_text: ', in_old_text, '\t in_new_text: ',in_new_text)

      else:

        all_occ = dict(zip(in_old_text,in_new_text))

        start = all_occ[pos.start]

        end = start+len(clean_text_data(old_text[pos]))#-1

        # check
        if clean_text_data(old_text[pos]) != new_text[slice(start, end)]:
          print('not similar', old_text[pos], '\t', new_text[slice(start, end)])


        return slice(start,end)


In [ ]:
def set_model_tokenize(model_name, text_data, dev_text_data ):
  model = model_name
  from transformers import AutoTokenizer
  tokenizer = AutoTokenizer.from_pretrained(model)

  tokenized = tokenizer(text_data,  is_split_into_words = False, return_offsets_mapping = True, add_special_tokens = True, truncation = True, padding = 'max_length', max_length=512, stride=128, return_overflowing_tokens=True,
  )  #when text is not splitted, the positions in str are accurate

  tokens = [tokenizer.convert_ids_to_tokens(tokenizedx) for tokenizedx in tokenized.input_ids]

  dev_tokenized = tokenizer(dev_text_data,  is_split_into_words = False,  return_offsets_mapping = True, add_special_tokens = True,  padding = 'max_length', truncation = True, max_length=512, stride=128, return_overflowing_tokens=True, )  #when text is not splitted, the positions are accurate

  dev_tokens = [tokenizer.convert_ids_to_tokens(tokenizedx) for tokenizedx in dev_tokenized.input_ids]

  return tokenized, dev_tokenized, tokens, dev_tokens

tokenized, dev_tokenized, tokens, dev_tokens = set_model_tokenize(model_name, text_data, dev_text_data )

In [ ]:
class Pre_processing:
    def __init__(self):

    def drop(self, data, drop_strategies):
        return self.data

    def fillna(self, ntrain, fill_strategies):
        def fill(column, fill_with):

    def feature_engineering(self):

    def label_encoder(self, columns):

    def get_dummies(self, columns):

    def norm_data(self, columns):


In [ ]:
class Pre_processing:
    """
    This class prepares the data berfore applying ML
    """
    def __init__(self):

        print()
        print('pre-processing object is created')
        print()

    def drop(self, data, drop_strategies):
        """
        This function is used to drop a column or row from the dataset.
        ...
        Attributes
        ----------
        data : Pandas DataFrame
            The data you want to drop data from.
        drop_strategies : A list of tuples, each tuple has the data to drop,
        and the axis(0 or 1)

        Returns
        ----------
        A new dataset after dropping the unwanted data.
        """

        self.data=data

        for columns, ax in drop_strategies:
            if len(columns)==1:
                self.data=self.data.drop(labels=column, axis=ax)
            else:
                for column in columns:
                    self.data=self.data.drop(labels=column, axis=ax)
        return self.data

    def fillna(self, ntrain, fill_strategies):
        """
        This function fills NA/NaN values in a specific column using a specified method(zero,mean,...)
        ...
        Attributes
        ----------
        data : Pandas DataFrame
            The data you want to impute its missing values
        fill_strategies : A dictionary, its keys represent the columns,
        and the values represent the value to use to fill the Nulls.

        Returns
        ----------
        A new dataset without null values.
        """
        def fill(column, fill_with):

                if str(fill_with).lower() in ['zero', 0]:
                    self.data[column].fillna(0, inplace=True)
                elif str(fill_with).lower()=='mode':
                    self.data[column].fillna(self.data[column].mode()[0], inplace=True)
                elif str(fill_with).lower()=='mean':
                    self.data[column].fillna(self.data[column].mean(), inplace=True)
                elif str(fill_with).lower()=='median':
                    self.data[column].fillna(self.data[column].median(), inplace=True)
                else:
                    self.data[column].fillna(fill_with, inplace=True)

                return self.data

        #LotFrontage: Linear feet of street connected to property
        self.data['LotFrontage'] = self.data.groupby('Neighborhood')['LotFrontage'].apply(lambda x: x.fillna(x.median())).values

        # Meaning that NO Masonry veneer
        self.data['MSZoning'] = self.data['MSZoning'].transform(lambda x: x.fillna(x.mode().values[0]))

        #imputing columns according to its strategy
        for columns, strategy in fill_strategies:
            if len(columns)==1:
                fill(columns[0], strategy)
            else:
                for column in columns:
                    fill(column, strategy)

        return self.data

    def feature_engineering(self):
        """
        This function is used to apply some feature engineering on the data.
        ...
        Attributes
        ----------
        data : Pandas DataFrame
            The data you want to apply feature engineering on.

        Returns
        ----------
        A new dataset with new columns and some additions.
        """
        # creating new columns
        self.data['TotalSF'] = self.data['TotalBsmtSF'] + self.data['1stFlrSF'] + self.data['2ndFlrSF']

        # Convert some columns from numeric to string
        self.data[['YrSold','MSSubClass','MoSold','OverallCond']] = self.data[['YrSold','MSSubClass','MoSold','OverallCond']].astype(str)

        # Convert some columns from numeric to int
        self.data[['BsmtHalfBath','BsmtFinSF1', 'BsmtFinSF2','BsmtFullBath','BsmtUnfSF','GarageCars','GarageArea']]\
        =self.data[['BsmtHalfBath','BsmtFinSF1', 'BsmtFinSF2','BsmtFullBath','BsmtUnfSF','GarageCars','GarageArea']].astype(int)

        return self.data

    def label_encoder(self, columns):
        """
        This function is used to encode the data to categorical values to benefit from increasing or
        decreasing to build the model
        ...
        Attributes
        ----------
        data : Pandas DataFrame
            The data you want to encode.
        columns : columns to convert.

        Returns
        ----------
        A dataset without categorical data.
        """

        # Convert all categorical collumns to numeric values
        lbl = LabelEncoder()

        self.data[columns] = self.data[columns].apply(lambda x:lbl.fit_transform(x.astype(str)).astype(int))

        return self.data

    def get_dummies(self, columns):
        """
        This function is used to convert the data to dummies values.
        ...
        Attributes
        ----------
        data : Pandas DataFrame
            The data you want to convert.

        Returns
        ----------
        A dataset with dummies.
        """

        # convert our categorical columns to dummies
        for col in columns:
            dumm = pd.get_dummies(self.data[col], prefix = col, dtype=int)
            self.data = pd.concat([self.data, dumm], axis=1)

        self.data.drop(columns, axis=1, inplace=True)

        return self.data

    def norm_data(self, columns):
        """
        This function is used to normalize the data.
        ...
        Attributes
        ----------
        data : Pandas DataFrame
            The data you want to normalize.

        Returns
        ----------
        A new normalized dataset.
        """

        # Normalize our numeric data
        self.data[columns] = self.data[columns].apply(lambda x:np.log1p(x)) #Normalize the data with Logarithms

        return self.data